# Автоматическая обработка текстов 
## Домашнее задание 1 [10 баллов] до 23:59 15.03.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе 
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]


Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание, с любого сервиса с прогнозом погоды или используя его API.
Примеры сервисов:
* gismeteo.ru, https://www.gismeteo.ru/api/ – Gismeteo, API Gismeteo 
* https://tech.yandex.ru/weather/ – API Yandex.Погоды
* https://sinoptik.com.ru – Sinoptik
* любой другой 

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальная температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

#### Попробую распарсить html с gismeteo руками

In [1]:
import requests
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

Получим код страницы

In [6]:
url = "https://gismeteo.ru/weather-dolgoprudny-12631/10-days/"
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36',
}

req = requests.get(url, headers=headers)

assert req.status_code == 200, 'request failed'
soup = BeautifulSoup(req.text, "lxml")

Посмотрим на код прогноза

In [52]:
for data in soup.findAll('div', attrs={'data-widget-id' : 'forecast'}):
    print(data)

<div class="widget js_widget" data-widget-id="forecast" data-widget-name="Погода"><div class="widget__body"><div class="widget__container"><div class="widget__row"><div class="widget__item"><div class="w_date"><a class="link blue weekend" href="/weather-dolgoprudny-12631/"><div class="w_date__day">Вс</div><span class="w_date__date weekend">
                                                                            4 мар
                                                </span></a></div></div><div class="widget__item"><div class="w_date"><a class="link blue" href="/weather-dolgoprudny-12631/tomorrow/"><div class="w_date__day">Пн</div><span class="w_date__date black">
                                                                            5
                                                </span></a></div></div><div class="widget__item"><div class="w_date"><a class="link blue" href="/weather-dolgoprudny-12631/3-day/"><div class="w_date__day">Вт</div><span class="w_date__date black">
  

Функция, вытаскивающая даты

In [50]:
def get_dates(soup):
    dates = []
    for date in soup.findAll('div', attrs={'class' : 'widget__item'}):
        try:
            day = date.find('span', attrs={'class': 'w_date__date'}).get_text().strip()
            weeks_day = date.find('div', attrs={'class': 'w_date__day'}).get_text().strip()
        except:
            break
        dates.append({
            "day": day,
            "weeks_day": weeks_day
        })
    return dates

In [51]:
dates = get_dates(soup)
print(*map(lambda d: "Day: {}, day of the week: {}".format(d['day'], d['weeks_day']), dates), sep="\n")

Day: 4 мар, day of the week: Вс
Day: 5, day of the week: Пн
Day: 6, day of the week: Вт
Day: 7, day of the week: Ср
Day: 8, day of the week: Чт
Day: 9, day of the week: Пт
Day: 10, day of the week: Сб
Day: 11, day of the week: Вс
Day: 12, day of the week: Пн
Day: 13, day of the week: Вт


Вытащим максимальный ветер

In [60]:
def get_wind(soup):
    winds = []
    for i, data in enumerate(soup.findAll('div', attrs={'class' : 'w_wind__warning w_wind__warning_'})):
        if i >= 10:
            break
        winds.append(data.get_text().strip())
    return winds

In [65]:
winds = get_wind(soup)
print(*map(lambda d: 
           "Day: {}, day of the week: {}, wind: {}".format(
               d[0]['day'], d[0]['weeks_day'], d[1]), zip(dates, winds)
          ), sep="\n")

Day: 4 мар, day of the week: Вс, wind: 12
Day: 5, day of the week: Пн, wind: 14
Day: 6, day of the week: Вт, wind: 8
Day: 7, day of the week: Ср, wind: 10
Day: 8, day of the week: Чт, wind: 11
Day: 9, day of the week: Пт, wind: 7
Day: 10, day of the week: Сб, wind: 5
Day: 11, day of the week: Вс, wind: 7
Day: 12, day of the week: Пн, wind: 6
Day: 13, day of the week: Вт, wind: 8


Вытащим температуры

In [75]:
def get_temperatures(soup):
    temps = []
    for i, temp in enumerate(soup.findAll('div', attrs={'class' : 'value'})):
        if i >= 10:
            break
        max_t = temp.find('div', attrs={'class' : 'maxt'}).get_text()
        min_t = temp.find('div', attrs={'class' : 'mint'}).get_text()
        temps.append({
            'max_t': max_t,
            'min_t': min_t
        })
    return temps

In [77]:
temps = get_temperatures(soup)
print(*map(lambda d: 
           "Day: {}; day of the week: {}; wind: {}; temps: max={}, min={}".format(
               d[0]['day'], d[0]['weeks_day'], d[1], d[2]['max_t'], d[2]['min_t']), zip(dates, winds, temps)
          ), sep="\n")

Day: 4 мар; day of the week: Вс; wind: 12; temps: max=−9, min=−11
Day: 5; day of the week: Пн; wind: 14; temps: max=−8, min=−12
Day: 6; day of the week: Вт; wind: 8; temps: max=−12, min=−18
Day: 7; day of the week: Ср; wind: 10; temps: max=−9, min=−17
Day: 8; day of the week: Чт; wind: 11; temps: max=−6, min=−10
Day: 9; day of the week: Пт; wind: 7; temps: max=−6, min=−8
Day: 10; day of the week: Сб; wind: 5; temps: max=−4, min=−8
Day: 11; day of the week: Вс; wind: 7; temps: max=−2, min=−8
Day: 12; day of the week: Пн; wind: 6; temps: max=+1, min=−8
Day: 13; day of the week: Вт; wind: 8; temps: max=−1, min=−5


Найдем осадки. Для этого нужны регулярки, ибо нулевые осадки имеют два класса, в отличии от не нулевых.

In [82]:
import re

In [90]:
precipitation_exp = re.compile(r"w_precipitation__value( w_precipitation__value_zero)?")

In [91]:
precipitations = ["w_precipitation__value", "w_precipitation__value w_precipitation__value_zero"]

In [92]:
for pre in precipitations:
    found_pre = precipitation_exp.match(pre).group()
    print("{}: {}".format(pre, pre == found_pre))

w_precipitation__value: True
w_precipitation__value w_precipitation__value_zero: True


In [96]:
def get_precipitations(soup):
    pres = []
    for pre in soup.findAll('div', attrs={'class' : precipitation_exp}):
        pres.append(pre.get_text().strip())
    return pres

In [97]:
pres = get_precipitations(soup)
print(*map(lambda d: 
           "Day: {}; day of the week: {}; wind: {}; temps: max={}, min={}; precipitations: {}".format(
               d[0]['day'], d[0]['weeks_day'], d[1], d[2]['max_t'], d[2]['min_t'], d[3]), zip(dates, winds, temps, pres)
          ), sep="\n")

Day: 4 мар; day of the week: Вс; wind: 12; temps: max=−9, min=−11; precipitations: 6,8
Day: 5; day of the week: Пн; wind: 14; temps: max=−8, min=−12; precipitations: 1,7
Day: 6; day of the week: Вт; wind: 8; temps: max=−12, min=−18; precipitations: 0
Day: 7; day of the week: Ср; wind: 10; temps: max=−9, min=−17; precipitations: 0,1
Day: 8; day of the week: Чт; wind: 11; temps: max=−6, min=−10; precipitations: 1,7
Day: 9; day of the week: Пт; wind: 7; temps: max=−6, min=−8; precipitations: 2,8
Day: 10; day of the week: Сб; wind: 5; temps: max=−4, min=−8; precipitations: 0,2
Day: 11; day of the week: Вс; wind: 7; temps: max=−2, min=−8; precipitations: 0
Day: 12; day of the week: Пн; wind: 6; temps: max=+1, min=−8; precipitations: 0
Day: 13; day of the week: Вт; wind: 8; temps: max=−1, min=−5; precipitations: 0,2


Запихаем эту всю инфу в pandas.DataFrame

In [98]:
import pandas as pd

In [103]:
data = [{
    'day': date['day'].replace('мар', '').strip(),
    'weeks_day': date['weeks_day'],
    'wind': wind,
    'max_t': temp['max_t'],
    'min_t': temp['min_t'],
    'pre': pre
} for date, wind, temp, pre in zip(dates, winds, temps, pres)]

In [104]:
df = pd.DataFrame(data)

In [106]:
df

,day,max_t,min_t,pre,weeks_day,wind
0,4,−9,−11,"6,8",Вс,12
1,5,−8,−12,"1,7",Пн,14
2,6,−12,−18,0,Вт,8
3,7,−9,−17,"0,1",Ср,10
4,8,−6,−10,"1,7",Чт,11
5,9,−6,−8,"2,8",Пт,7
6,10,−4,−8,"0,2",Сб,5
7,11,−2,−8,0,Вс,7
8,12,+1,−8,0,Пн,6
9,13,−1,−5,"0,2",Вт,8


In [107]:
df.to_csv("data.csv")

### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на первые 10 дней февраля в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 км/час в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 мм за 7 дней*
    * *Выпадет 10 см снега за ближайшие 7 дней * 


Вместо НазваниеГорода и дней недели подставьте название выбранного вами города и дни недели, используя фунцкии для согласования существительных с предлогами. Используйте функции для согласования числительного с существительным для согласования длительности промежутков времени и слова "день" и чисел  с последующими единицами измерения.

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
word = 'яблоко'
parsed_word = morph.parse(word)[0]
print(1, parsed_word.make_agree_with_number(1).word) # согласование слова с числительным 1
print(2, parsed_word.make_agree_with_number(2).word) # согласование слова с числительным 2
print(3, parsed_word.make_agree_with_number(5).word) # согласование слова с числительным 5

1 яблоко
2 яблока
3 яблок


In [2]:
print(parsed_word.inflect({'gent'}).word) # слово в родительном падеже

яблока


Ниже приведен пример решения аналогичной задачи генерации текстов: генератор отчета о том, сколько и каких фруктов съел Вася.

In [17]:
def eating(n, fruit):
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join(['Вася съел', str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [18]:
eating(5, 'яблоко')

'Вася съел 5 яблок .'

In [19]:
eating(4, 'груша')

'Вася съел 4 груши .'

Пример более сложного генератора: фрукты могут есть не только мальчики, но и девочки, то есть, нужно не только согласовать числительное с существительным, но и поставить глагол в нужную форму.

In [27]:
def eating(name, n, fruit):
    name_tag = morph.parse(name)[0].tag
    if 'masc' in name_tag:
        gender = 'masc'
    if 'femn' in name_tag:
        gender = 'femn'
    verb = morph.parse("съесть")[0].inflect({'perf', gender,'sing','past','indc'}).word
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join([name, verb, str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [28]:
eating('Маша', 3, 'вишня')

'Маша съела 3 вишни .'

#### Нуус, попробуем

In [129]:
import pymorphy2

In [109]:
week2str = {
    'Пн': 'понедельник',
    'Вт': 'вторник',
    'Ср': 'среда',
    'Чт': 'четверг',
    'Пт': 'пятница',
    'Сб': 'суббота',
    'Вс': 'воскресенье'
}

city = 'Долгопрудный'

#### <i>Шаблон первый:</i>

В <i>день1(число)</i> в городе О похолодает/потеплеет на Х градус (-а, -ов) по сравнению с <i>день2(число)</i>

In [115]:
def parse_temp(temp):
    if temp  == '0':
        return 0
    return 1 * int(temp[1:]) if temp[0] == '+' else -1 * int(temp[1:])

In [125]:
def get_temp_delta(df, idx_day_1, idx_day_2):
    return (parse_temp(df.iloc[idx_day_1]['max_t']) + parse_temp(df.iloc[idx_day_1]['min_t']) - 
            parse_temp(df.iloc[idx_day_2]['max_t']) - parse_temp(df.iloc[idx_day_2]['min_t']))/2

In [181]:
def pattern_1(df, city, idx_day_1, idx_day_2):
    delta = int(get_temp_delta(df, idx_day_1, idx_day_2))
    if delta == 0:
        text = "В {}({}) в городе {} погода такая же как и в {}({})".format(
            week2str[df.iloc[idx_day_1]['weeks_day']], df.iloc[idx_day_1]['day'], city,
            morph.parse(week2str[df.iloc[idx_day_2]['weeks_day']])[0].inflect({'accs'}).word,
            df.iloc[idx_day_2]['day'],
        )
    else:
        if delta > 0:
            warm = 'потеплело'
        else:
            warm = 'похолодало'
        delta = abs(delta)
        degree = morph.parse('градус')[0].make_agree_with_number(delta).word
        text = "В {}({}) в городе {} {} на {} {} по сравнению с {}({})".format(
            week2str[df.iloc[idx_day_1]['weeks_day']], df.iloc[idx_day_1]['day'],
            city, warm, str(delta), degree,
            morph.parse(week2str[df.iloc[idx_day_2]['weeks_day']])[0].inflect({'ablt'}).word, 
            df.iloc[idx_day_2]['day']
        )
    return text

In [148]:
df

,day,max_t,min_t,pre,weeks_day,wind
0,4,−9,−11,"6,8",Вс,12
1,5,−8,−12,"1,7",Пн,14
2,6,−12,−18,0,Вт,8
3,7,−9,−17,"0,1",Ср,10
4,8,−6,−10,"1,7",Чт,11
5,9,−6,−8,"2,8",Пт,7
6,10,−4,−8,"0,2",Сб,5
7,11,−2,−8,0,Вс,7
8,12,+1,−8,0,Пн,6
9,13,−1,−5,"0,2",Вт,8


In [182]:
print(pattern_1(df, city, 1, 0))
print(pattern_1(df, city, 2, 1))
print(pattern_1(df, city, 3, 2))
print(pattern_1(df, city, 4, 3))
print(pattern_1(df, city, 5, 4))

В понедельник(5) в городе Долгопрудный погода такая же как и в воскресенье(4)
В вторник(6) в городе Долгопрудный похолодало на 5 градусов по сравнению с понедельником(5)
В среда(7) в городе Долгопрудный потеплело на 2 градуса по сравнению с вторником(6)
В четверг(8) в городе Долгопрудный потеплело на 5 градусов по сравнению с средой(7)
В пятница(9) в городе Долгопрудный потеплело на 1 градус по сравнению с четвергом(8)


#### <i>Шаблон в второй:</i>

Скорость ветра изменится на Х м/с в <i>день1(число)</i> по сравнению с <i>день2(число)</i>

In [152]:
def get_wind_delta(df, idx_day_1, idx_day_2):
    return int(df.iloc[idx_day_1]['wind']) - int(df.iloc[idx_day_2]['wind'])

In [159]:
def pattern_2(df, idx_day_1, idx_day_2):
    delta = get_wind_delta(df, idx_day_1, idx_day_2)
    if delta == 0:
        text = "В {}({}) скорость ветра такая же как и в {}({})".format(
            week2str[df.iloc[idx_day_1]['weeks_day']], df.iloc[idx_day_1]['day'],
            morph.parse(week2str[df.iloc[idx_day_2]['weeks_day']])[0].inflect({'accs'}).word,
            df.iloc[idx_day_2]['day'],
        )
    else:
        text = "Скорость ветра изменится на {} м/с в {}({}) по сравнению с {}({})".format(
            str(delta), week2str[df.iloc[idx_day_1]['weeks_day']], df.iloc[idx_day_1]['day'],
            morph.parse(week2str[df.iloc[idx_day_2]['weeks_day']])[0].inflect({'ablt'}).word, 
            df.iloc[idx_day_2]['day']
        )
    return text

In [160]:
df

,day,max_t,min_t,pre,weeks_day,wind
0,4,−9,−11,"6,8",Вс,12
1,5,−8,−12,"1,7",Пн,14
2,6,−12,−18,0,Вт,8
3,7,−9,−17,"0,1",Ср,10
4,8,−6,−10,"1,7",Чт,11
5,9,−6,−8,"2,8",Пт,7
6,10,−4,−8,"0,2",Сб,5
7,11,−2,−8,0,Вс,7
8,12,+1,−8,0,Пн,6
9,13,−1,−5,"0,2",Вт,8


In [161]:
print(pattern_2(df, 1, 0))
print(pattern_2(df, 2, 1))
print(pattern_2(df, 3, 2))
print(pattern_2(df, 4, 3))
print(pattern_2(df, 5, 4))

Скорость ветра изменится на 2 м/с в понедельник(5) по сравнению с воскресеньем(4)
Скорость ветра изменится на -6 м/с в вторник(6) по сравнению с понедельником(5)
Скорость ветра изменится на 2 м/с в среда(7) по сравнению с вторником(6)
Скорость ветра изменится на 1 м/с в четверг(8) по сравнению с средой(7)
Скорость ветра изменится на -4 м/с в пятница(9) по сравнению с четвергом(8)


#### <i>Шаблон в третий:</i>

За Х <i>день(дней)</i> выпало Y мм осадков

In [174]:
def str2float(s):
    return float('.'.join(s.split(',')))
    
def get_pre_delta(df, idx_day_1, idx_day_2):
    return sum([str2float(df.iloc[i]['pre']) for i in range(idx_day_1, idx_day_2 + 1)])

In [175]:
def pattern_3(df, idx_day_1, idx_day_2=None):
    if idx_day_2 is None:
        idx_day_2 = idx_day_1
    if idx_day_1 > idx_day_2:
        idx_day_1, idx_day_2 = idx_day_2, idx_day_1
    delta = get_pre_delta(df, idx_day_1, idx_day_2)
    n_days = idx_day_2 - idx_day_1 + 1
    days_str = morph.parse('день')[0].make_agree_with_number(n_days).word
    if delta == 0:
        text = "За {} {} осадков не выпало".format(n_days, days_str)
    else:
        text = "За {} {} выпало {} мм осадков".format(n_days, days_str, delta)
    return text

In [176]:
df

,day,max_t,min_t,pre,weeks_day,wind
0,4,−9,−11,"6,8",Вс,12
1,5,−8,−12,"1,7",Пн,14
2,6,−12,−18,0,Вт,8
3,7,−9,−17,"0,1",Ср,10
4,8,−6,−10,"1,7",Чт,11
5,9,−6,−8,"2,8",Пт,7
6,10,−4,−8,"0,2",Сб,5
7,11,−2,−8,0,Вс,7
8,12,+1,−8,0,Пн,6
9,13,−1,−5,"0,2",Вт,8


In [180]:
print(pattern_3(df, 1, 0))
print(pattern_3(df, 2, 5))
print(pattern_3(df, 3, 3))
print(pattern_3(df, 7, 7))
print(pattern_3(df, 4, 9))
print(pattern_3(df, 5, 9))

За 2 дня выпало 8.5 мм осадков
За 4 дня выпало 4.6 мм осадков
За 1 день выпало 0.1 мм осадков
За 1 день осадков не выпало
За 6 дней выпало 4.9 мм осадков
За 5 дней выпало 3.2 мм осадков


### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
    - <b>Ответ:</b> 
        - в навигаторе(через n метров/метра поверните направо/налево)
        - бот для сообщения температуры в квартире/времени до электрички
        - какие-нибудь оповещения для биржевых игроков
        - да много где
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 
    - <b>Ответ:</b> 
        - добавить число и месяц, между которыми идет сравнение 
        - выкачивать данные из архива и говорить, что было в прошлом(перевести некоторые слова в прошедее/настоящее/будущее время в зависимости от даты запроса)
        - если температура в рассматриваемый день < 0, то говорим, что выпало столько то снега, иначе дождя
        - итд итп

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 15.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку. Плагиат наказывается нулём баллов за задание.

При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям – Антону Емельянову. 

Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.


### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт 0pobDsj (для всех групп). 


 У вас появится курс “МФТИ > Автоматическая обработка текстов (2018)” в разделе “Посещает курсы”.
* Перейдите по ссылке “МФТИ > Автоматическая обработка текстов (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/325